In [1]:
from model import ModelConfig, LlamaModel
from train import TrainerConfig, DataLoader, Trainer

from transformers import AutoTokenizer

In [2]:
tokenizer_id = "HuggingFaceTB/SmolLM2-135M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token


In [4]:
model_config = ModelConfig(
    vocab_size=tokenizer.vocab_size,
    d_model=576,
    d_head=64,
    d_mlp_proj=1536,
    n_layers=30,
    n_kv_heads=3,
    n_attn_heads=9,
    rms_norm_eps=1e-5,
    initializer_range=0.041666666666666664,
    rope_theta=100000.0,
    padding_idx=tokenizer.pad_token_id
)

In [5]:
train_config = TrainerConfig(
    per_device_train_batch_size=8,
    max_seq_len=2048,
    num_epochs=50,
    learning_rate=1e-3,
    log_dir="runs/shakespeare"
)

In [6]:
with open("data/tiny_shakespeare.txt") as f:
    text = f.read()

In [7]:
model = LlamaModel(model_config)
dataloader = DataLoader(train_config, tokenizer, text=text)
trainer = Trainer(train_config, model)

Total train tokens             | 342,016
Num Trainable Params           | 162,826,560
Train device                   | cuda, NVIDIA GeForce RTX 3090, N=1
Training precision             | torch.bfloat16
Flash Attention                | True
torch.compile()                | True





In [8]:
trainer.train(dataloader)

Training steps                 | 210 
Step: 0, Training Loss: 11.39692, Tokens/sec: 1496.3612595824752
Step: 1, Training Loss: 9.98063, Tokens/sec: 1642.3986627833508
Step: 2, Training Loss: 12.49225, Tokens/sec: 82497.22358627409
Step: 3, Training Loss: 8.97854, Tokens/sec: 82210.1207514448
Step: 4, Training Loss: 8.53118, Tokens/sec: 87240.43839728912
Step: 5, Training Loss: 7.97263, Tokens/sec: 81196.92795908762
Step: 6, Training Loss: 7.82696, Tokens/sec: 80414.78203718596
Step: 7, Training Loss: 7.15195, Tokens/sec: 81210.9544627696
Step: 8, Training Loss: 6.98927, Tokens/sec: 80305.29658373963
Step: 9, Training Loss: 6.85874, Tokens/sec: 84987.0998709118
Step: 10, Training Loss: 6.70656, Tokens/sec: 86289.6941476574
Step: 11, Training Loss: 6.63914, Tokens/sec: 88933.12336839839
Step: 12, Training Loss: 6.70161, Tokens/sec: 85474.9501900465
Step: 13, Training Loss: 6.65715, Tokens/sec: 87528.98235635644
Step: 14, Training Loss: 6.73809, Tokens/sec: 88767.38323373672
Step: 15, Tra

In [9]:
input_ids = tokenizer(["HAMLET:\nTo be or"], return_tensors="pt")['input_ids'].to(trainer.device)
idx = model.generate(input_ids, temperature=0.25, top_k=50, max_new_tokens=64)
print(tokenizer.batch_decode(idx)[0])

HAMLET:
To be or the be
' the the the a the the a have
I,


I,

And this,


Ber,
I,



I,



I,


And the a the the a the be the be the the me,

